# Web Scraping of RSA Conference USA 2019
### David Lowe
### July 14, 2019

SUMMARY: The purpose of this project is to practice web scraping by extracting specific pieces of information from a website. The web scraping python code leverages the BeautifulSoup module.

INTRODUCTION: On occasions we have a need to download a batch of documents off a single web page without clicking on the download link one at a time. This web scraping script will automatically traverse through the entire web page and collect all links to the PDF and PPTX documents. The script will also download the documents as part of the scraping process.

Starting URLs: https://www.rsaconference.com/events/us19/presentations?type=presentations

## Loading Libraries and Packages

In [1]:
# Refresh package repositories and set up additional Linux and Python packages
!apt-get update
!apt install chromium-chromedriver
!pip install -q pymysql selenium

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (144 kB/s)
Reading package lists... Done
Reading package lists... Done
Building de

In [0]:
import numpy as np
import pandas as pd
import os
import shutil
import smtplib
import sys
from email.message import EmailMessage
from datetime import datetime
import requests
from requests.exceptions import HTTPError
from requests.exceptions import ConnectionError
from bs4 import BeautifulSoup
from random import randint
from time import sleep
from selenium import webdriver
import pymysql

startTimeScript = datetime.now()

## Setting up the basic functions

In [0]:
def email_notify(msg_text):
    sender = os.environ.get('MAIL_SENDER')
    receiver = os.environ.get('MAIL_RECEIVER')
    gateway = os.environ.get('SMTP_GATEWAY')
    smtpuser = os.environ.get('SMTP_USERNAME')
    password = os.environ.get('SMTP_PASSWORD')
    if sender==None or receiver==None or gateway==None or smtpuser==None or password==None:
        sys.exit("Incomplete email setup info. Script Processing Aborted!!!")
    msg = EmailMessage()
    msg.set_content(msg_text)
    msg['Subject'] = 'Notification from Python Web Scraping Script'
    msg['From'] = sender
    msg['To'] = receiver
    server = smtplib.SMTP(gateway, 587)
    server.starttls()
    server.login(smtpuser, password)
    server.send_message(msg)
    server.quit()

In [0]:
def download_file(doc_path):
#    local_file = os.path.basename(doc_path)
    local_file = doc_path.split('/')[-1]
    gdrivePrefix = '/content/gdrive/My Drive/Colab_Downloads/'
    dest_file = gdrivePrefix + local_file
    with requests.get(doc_path, stream=True) as r:
        with open(dest_file, 'wb') as f:
            shutil.copyfileobj(r.raw, f)
    print('Downladed file: ' + dest_file)

## Setting up the necessary parameters

In [5]:
# Mount Google Drive locally

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Set up the verbose flag to print detailed messages for debugging (setting True will activate!)
verbose = False

# Set up the sendNotification flag to send progress emails (setting True will send emails!)
sendNotification = False

# Set up the executeDownload flag to download files (setting True will download!)
executeDownload = True

In [0]:
if (sendNotification): email_notify("The web scraping process has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [0]:
# Specifying the URL of desired web page to be scrapped
startingURL = "https://www.rsaconference.com/events/us19/presentations?type=presentations"
websiteURL = "https://www.rsaconference.com"

# Creating an html document from the URL
uastring = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:67.0) Gecko/20100101 Firefox/67.0"
headers={'User-Agent': uastring}

## Performing the Scraping and Processing

In [0]:
if (sendNotification): email_notify("The web page loading and item extraction process has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [10]:
try:
    s = requests.Session()
    resp = s.get(startingURL, headers=headers)
    if (verbose): print(resp.text)
except HTTPError as e:
    print('The server could not serve up the web page!')
    sys.exit("Script processing cannot continue!!!")
except ConnectionError as e:
    print('The server could not be reached due to connection issues!')
    sys.exit("Script processing cannot continue!!!")

if (resp.status_code==requests.codes.ok):
    print('Successfully accessed the web page: ' + startingURL)
    webPage = BeautifulSoup(resp.text, 'lxml')

Successfully accessed the web page: https://www.rsaconference.com/events/us19/presentations?type=presentations


In [11]:
# Gather all links to the document
collection = webPage.find_all("a")
i = 0

for item in collection:
    if (verbose): print(item)
    docPath = item['href']
    if docPath.lower().endswith(".pdf") | docPath.lower().endswith(".pptx") | docPath.lower().endswith(".zip"):
        i = i + 1
        docPath = websiteURL + docPath
        # Adding random wait time so we do not hammer the website needlessly
        waitTime = randint(2,5)
        print("Waiting " + str(waitTime) + " seconds to retrieve " + docPath)
        sleep(waitTime)
        if (executeDownload): download_file(docPath)

print('Finished finding all available documents on the web page!')

Waiting 2 seconds to retrieve https://www.rsaconference.com/writable/presentations/file_upload/str-f03-10-lies-you-tell-your-security-vendors-and-what-to-do-instead.pdf
Downladed file: /content/gdrive/My Drive/Colab_Downloads/str-f03-10-lies-you-tell-your-security-vendors-and-what-to-do-instead.pdf
Waiting 5 seconds to retrieve https://www.rsaconference.com/writable/presentations/file_upload/idy-f02-12-ways-to-hack-2fa.pdf
Downladed file: /content/gdrive/My Drive/Colab_Downloads/idy-f02-12-ways-to-hack-2fa.pdf
Waiting 3 seconds to retrieve https://www.rsaconference.com/writable/presentations/file_upload/tech-t07-2028-future-state-long-live-the-firewall-final.pdf
Downladed file: /content/gdrive/My Drive/Colab_Downloads/tech-t07-2028-future-state-long-live-the-firewall-final.pdf
Waiting 4 seconds to retrieve https://www.rsaconference.com/writable/presentations/file_upload/idy-f01-4g-to-5g-evolution-in-depth-security-perspective.pdf
Downladed file: /content/gdrive/My Drive/Colab_Downloads

In [12]:
print('Number of documents processed:', i)

Number of documents processed: 466


In [0]:
if (sendNotification): email_notify("The web scraping process has completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [14]:
print ('Total time for the script:',(datetime.now() - startTimeScript))

Total time for the script: 0:38:21.758038
